<a href="https://colab.research.google.com/github/Stevenn9981/tic_tac_toe/blob/master/tic_tac_toe_part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!sudo apt-get update
!sudo apt-get install -y xvfb ffmpeg freeglut3-dev
!pip install imageio
!pip install pyvirtualdisplay
!pip install tf-agents[reverb]
!pip install pyglet

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [47.6 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [632 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,494 kB]
Get:13 http://

In [2]:
import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay
import reverb
import random
import os
import shutil
import tempfile
import zipfile
import copy

import pygame as pg
from pygame import gfxdraw
from pygame.locals import *

import tensorflow as tf

from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import py_driver
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import sequential
from tf_agents.networks import q_network
from tf_agents.networks import categorical_q_network
from tf_agents.policies import policy_saver
from tf_agents.policies import py_tf_eager_policy
from tf_agents.policies import random_tf_policy
from tf_agents.policies import random_py_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.replay_buffers import reverb_replay_buffer
from tf_agents.replay_buffers import reverb_utils
from tf_agents.trajectories import trajectory
from tf_agents.trajectories import PolicyStep
from tf_agents.specs import tensor_spec
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.trajectories import time_step as ts, TimeStep

import gym
import numpy as np
import math
from tabulate import tabulate

from typing import Tuple, List


tempdir = "./"

pygame 2.1.3 (SDL 2.0.22, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
tf.version.VERSION

'2.14.0'

## Hyper-parameters

In [4]:
num_iterations = 1200  # @param {type:"integer"}

initial_collect_steps = 100  # @param {type:"integer"}
initial_collect_episodes = 5  # @param {type:"integer"}
collect_steps_per_iteration = 1  # @param {type:"integer"}
collect_episodes_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 10000  # @param {type:"integer"}

batch_size = 128  # @param {type:"integer"}
learning_rate = 5e-4  # @param {type:"number"}
log_interval = 5  # @param {type:"integer"}

num_eval_episodes = 50  # @param {type:"integer"}
eval_interval = 20  # @param {type:"integer"}

gamma = 0  # @param {type:"number"}
n_step_update = 1  # @param {type:"integer"}
fc_layer_params = (100,)

BOARD_SIZE = 9  # @param {type:"integer"}

REWARD_DRAW = 0  # @param {type:"number"}
REWARD_ALIVE = 0  # @param {type:"number"}
REWARD_NON_ADJ = -0.03  # @param {type:"number"}

# '_' means empty position, 'O' and 'X' means two players.
REWARD_ACTIVE_TWO = 0.1  # @param {type:"number"} _OO_ or _XX_, we call it active_two
REWARD_NONACT_THREE = 0.3  # @param {type:"number"} _OOOX or _XXXO or XOOO_ or OXXX_, we call it non_active_three
REWARD_ACTIVE_THREE = 0.9  # @param {type:"number"} _OOO_ or _XXX_, we call it active_three
REWARD_WIN = 3  # @param {type:"number"} _OOOO_ or _XXXX_

tempdir = "./"

## Environment

In [5]:
class TicTacToeEnv2(py_environment.PyEnvironment):
    """
    Implementation of a TicTacToe Environment based on the instructions of Part 2, Question 1.
    """

    def __init__(self, train=False) -> None:
        """This class contains a TicTacToe environment for Part 2

        Args:
            train (bool): whether this is an environment for training.
        """

        self.ener_bin_len = 0.2
        self.num_bin = int(1 / self.ener_bin_len) + 1
        self.n_actions = BOARD_SIZE * BOARD_SIZE * self.num_bin  # 9 * 9 * self.num_bin actions

        self._observation_spec = {
            'state': array_spec.BoundedArraySpec(shape=(BOARD_SIZE, BOARD_SIZE, 5), dtype=np.int_, minimum=0,
                                                 maximum=1),
            'legal_moves': array_spec.ArraySpec(shape=(self.n_actions,), dtype=np.bool_)}

        self._action_spec = array_spec.BoundedArraySpec(shape=(), dtype=np.int_, minimum=0, maximum=self.n_actions - 1)
        self.colors = [1, 2]
        self.screen = None
        self.fields_per_side = BOARD_SIZE
        self.train = train
        self.reset()

    def observation_spec(self):
        return self._observation_spec

    def action_spec(self):
        return self._action_spec

    def get_result(self):
        return self.result

    def _reset(self) -> TimeStep:
        """
        reset the board game and state
        """
        self.board: np.ndarray = np.zeros(
            (self.fields_per_side, self.fields_per_side), dtype=int
        )
        self.current_player = 1
        self.info = {"players": {1: {"energy": 10}, 2: {"energy": 10}}, "Occupied": set(),
                     "legal_moves": np.ones((self.n_actions,), dtype=bool)}
        self.latest_action = None

        # 0 means not finished, 1 or 2 means the winner and 3 means draw
        self.result = 0

        # return self.decompose_board_to_state()
        observations_and_legal_moves = {'state': self.decompose_board_to_state(),
                                        'legal_moves': self.info["legal_moves"]}
        return ts.restart(observations_and_legal_moves)

    def if_chess_nearby(self, row, col) -> bool:
        """
        Determine whether there are chess pieces in 2 squares around the given position (row, col)

        Args:
          row (int): row index
          col (int): column index

        Returns:
          res (boolean): true, if there are
        """
        for i in range(-2, 3):
            for j in range(-2, 3):
                if 0 <= row + i < BOARD_SIZE and 0 <= col + j < BOARD_SIZE:
                    if self.board[row + i, col + j] != 0:
                        return True
        return False

    def decompose_board_to_state(self):
        """
        Our state is a 9x9x5 matrix.
        The first layer is the opponent's play history, 0 means no stone, 1 means stones placed by the opponent.
        The second layer is the current player's history, 0 means no stone, 1 means stones placed by the current player.
        The third layer is the opponent's latest play-out; only one entry is 1 and the others are 0. If the board is empty now, all entries are 0.
        The fourth layer is whether the current player is the first hand. an array that is full of 1 means yes, and 0 means no.
        The fifth layer shows the empty positions whose adjacent positions are not all empty. 1 means there is at least one chess piece in its adjacent positions, and 0 means no.
        """
        opponent = 2 if self.current_player == 1 else 1
        o_plays = (self.board == opponent) * 1
        c_plays = (self.board == self.current_player) * 1
        l_play = np.zeros_like(self.board)
        if self.latest_action:
            r, c, _ = self.decode_action(self.latest_action)
            l_play[r, c] = 1
        if_first = np.full_like(self.board, (self.current_player == 1) * 1)
        adj_play = np.zeros_like(self.board)
        for row in range(adj_play.shape[0]):
            for col in range(adj_play.shape[1]):
                if self.board[row, col] == 0 and self.if_chess_nearby(row, col):
                    adj_play[row, col] = 1
        return np.stack([o_plays, c_plays, l_play, if_first, adj_play], axis=2)

    def _step(self, position: int) -> Tuple[np.ndarray, int, bool, dict]:
        """step function of the TicTacToeEnv2

        Args:
          position (int): integer between [0, 80], each representing a field on the board

        Returns:
          state (np.array): state of 2 players' history, 0 means no stone, 1 means stones placed by the corresponding player (shape: 9x9x2).
          reward (int): reward of the currrent step
          done (boolean): true, if the game is finished
          (dict): empty dict for future game related information
        """
        position = int(position)
        if not (0 <= position < self.n_actions):
            raise ValueError(f"action '{position}' is not in action_space")

        reward = REWARD_ALIVE
        (row, col, ener) = self.decode_action(position)
        energy_to_use = ener * self.ener_bin_len

        # If the agent/player does not choose an empty square, raise the ValueError.
        if self.board[row, col] != 0:
            if len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE:
                raise ValueError('BORAD IS FULL!')
            raise ValueError('ERROR: Not A LEGAL MOVE (NOT EMPTY)')

        # Check whether there is enough energy to use for the current player
        if self.info["players"][self.current_player]["energy"] < energy_to_use:
            energy_to_use = self.info["players"][self.current_player]["energy"]
        self.info["players"][self.current_player]["energy"] -= energy_to_use

        # randomly select an adjacent position with probability 1/8 * (8 / 9 - 6 / 9 * energy_to_use)
        if random.random() < (1 - 1 / 9 - 6 / 9 * energy_to_use):
            row, col = self.choose_adj_pos(row, col)

        # if len(self.info["Occupied"]) != 0 and not self.if_chess_nearby(row, col):
        #     reward += REWARD_NON_ADJ

        win = False
        if 0 <= row < BOARD_SIZE and 0 <= col < BOARD_SIZE and self.board[row, col] == 0:
            self.board[row, col] = self.current_player  # drop the piece on the field
            win = self._is_win(self.current_player, row, col)

            cnt_act_two = self.detect_alive_two(row, col)
            cnt_non_act_three, cnt_act_three = self.detect_three(row, col)

            reward += (
                    cnt_act_two * REWARD_ACTIVE_TWO + cnt_non_act_three * REWARD_NONACT_THREE + cnt_act_three * REWARD_ACTIVE_THREE)

            position = row * BOARD_SIZE + col
            num_places = BOARD_SIZE * BOARD_SIZE

            self.latest_action = int(position + energy_to_use / self.ener_bin_len * num_places)
            for i in range(self.num_bin):
                action = position + i * num_places
                self.info["Occupied"].add(action)
                self.info['legal_moves'][action] = False
        elif self.latest_action:
            r, c, _ = self.decode_action(self.latest_action)
            self.latest_action = int(r * BOARD_SIZE + c + energy_to_use / self.ener_bin_len * BOARD_SIZE * BOARD_SIZE)

        if win:
            self.result = self.current_player
            reward += REWARD_WIN
        elif len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE:  # Draw
            self.result = 3
            reward += REWARD_DRAW

        done = (win or len(self.info["Occupied"]) == BOARD_SIZE * BOARD_SIZE)
        self.current_player = self.current_player + 1 if self.current_player == 1 else 1
        state = self.decompose_board_to_state()

        observations_and_legal_moves = {'state': state, 'legal_moves': self.info['legal_moves']}

        if done:
            return ts.termination(observations_and_legal_moves, reward)
        else:
            return ts.transition(observations_and_legal_moves, reward)

    def detect_alive_two(self, row: int, col: int) -> int:
        """ Detect how many alive_two can obtain by this play out.

        Args:
            row (int): row of the current play
            col (int): column of the current play

        Returns:
            cnt (int): the number of alive_two
        """
        cnt = 0
        adjs = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [0, -1], [1, -1], [1, 0], [1, 1]]
        for adj in adjs:
            p1_r, p1_c = row + 2 * adj[0], col + 2 * adj[1]
            p2_r, p2_c = row - adj[0], col - adj[1]
            if 0 <= p1_r < 9 and 0 <= p1_c < 9 and 0 <= p2_r < 9 and 0 <= p2_c < 9:
                if self.board[row + adj[0], col + adj[1]] == self.current_player and self.board[p1_r, p1_c] == 0 and \
                        self.board[p2_r, p2_c] == 0:
                    cnt += 1
        return cnt

    def detect_three(self, r: int, c: int) -> tuple:
        """ Detect how many non_active_three and active_three can obtain by this play out.

        Args:
            r (int): row of the current play
            c (int): column of the current play

        Returns:
            cnt_non_act (int): the number of non_active_three
            cnt_act (int): the number of active_three
        """
        cnt_non_act = 0
        cnt_act = 0
        opponent = self.current_player + 1 if self.current_player == 1 else 1
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        for direct in directions:
            count = 0
            for offset in range(-2, 3):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if self.board[r + offset * direct[0], c + offset * direct[1]] == self.current_player:
                        count += 1
                        if count == 3:
                            p1_r, p1_c = r + (offset + 1) * direct[0], c + (offset + 1) * direct[1]
                            p2_r, p2_c = r + (offset - 3) * direct[0], c + (offset - 3) * direct[1]

                            p1_is_empty = (0 <= p1_r < 9 and 0 <= p1_c < 9 and self.board[p1_r, p1_c] == 0)
                            p2_is_empty = (0 <= p2_r < 9 and 0 <= p2_c < 9 and self.board[p2_r, p2_c] == 0)

                            if p1_is_empty and p2_is_empty:
                                cnt_act += 1
                            elif p1_is_empty or p2_is_empty:
                                cnt_non_act += 1
                            break
                    else:
                        count = 0

        return cnt_non_act, cnt_act

    def choose_adj_pos(self, row: int, col: int) -> tuple:
        """ Randomly select an adjacent position with equal probabilities.

        Args:
            row (int): row of the current play
            col (int): column of the current play

        Returns:
            row (int): row of the selected adjacent position
            col (int): column of the selected adjacent position
        """

        adjs = [[-1, -1], [-1, 0], [-1, 1], [0, 1], [0, -1], [1, -1], [1, 0], [1, 1]]
        adj = random.choice(adjs)
        row, col = row + adj[0], col + adj[1]
        return row, col

    def _is_win(self, color: int, r: int, c: int) -> bool:
        """check if this player results in a winner

        Args:
            color (int): of the player
            r (int): row of the current play
            c (int): column of the current play

        Returns:
            bool: indicating if there is a winner
        """

        # check if four equal stones are aligned (horizontal, verical or diagonal)
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        for direct in directions:
            count = 0
            for offset in range(-3, 4):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if self.board[r + offset * direct[0], c + offset * direct[1]] == color:
                        count += 1
                        if count == 4:
                            return True
                    else:
                        count = 0

        return False

    def decode_action(self, action: int) -> List[int]:
        """decode the action integer into a colum and row value

        0 = upper left corner
        8 = lower right corner

        Args:
            action (int): action

        Returns:
            List[int, int]: a list with the [row, col] values
        """
        action = np.clip(action, 0, self.n_actions)

        num_places = BOARD_SIZE * BOARD_SIZE

        board_position = action % num_places
        energy = action // num_places

        col = board_position % BOARD_SIZE
        row = board_position // BOARD_SIZE
        assert 0 <= col < BOARD_SIZE
        assert 0 <= row < BOARD_SIZE
        assert 0 <= energy < self.num_bin
        return [row, col, energy]

    def render(self, render_mode="rgb_array") -> np.ndarray:
        """Render the board
        Print a string that shows the current board, if render_mode == human,
        Return the RGB array of a figure which shows the current board.
        """
        board = np.zeros((BOARD_SIZE, BOARD_SIZE), dtype=str)
        for ii in range(BOARD_SIZE):
            for jj in range(BOARD_SIZE):
                if self.board[ii, jj] == 0:
                    board[ii, jj] = "-"
                elif self.board[ii, jj] == 1:
                    board[ii, jj] = "X"
                elif self.board[ii, jj] == 2:
                    board[ii, jj] = "O"

        if render_mode == "human":
            board = tabulate(board, tablefmt="fancy_grid")
            print(board)
            print("\n")

        width = height = 400

        white = (255, 255, 255)
        line_color = (0, 0, 0)
        red = (255, 0, 0)

        os.environ["SDL_VIDEODRIVER"] = "dummy"
        pg.init()

        # Set up the drawing window
        if self.screen is None:
            self.screen = pg.display.set_mode([width + 272, height + 16])

        self.screen.fill(white)
        # drawing vertical lines
        for i in range(10):
            pg.draw.line(self.screen, line_color, (width / BOARD_SIZE * i, 0), (width / BOARD_SIZE * i, height), 2)

        # drawing horizontal lines
        for i in range(10):
            pg.draw.line(self.screen, line_color, (0, height / BOARD_SIZE * i), (width, height / BOARD_SIZE * i), 2)
        pg.display.flip()

        latest_row, latest_col, latest_enr = -1, -1, 0
        if self.latest_action:
            latest_row, latest_col, latest_enr = self.decode_action(self.latest_action)

        # drawing noughts and crosses
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                color = line_color
                if latest_row == i and latest_col == j:
                    color = red
                if self.board[i, j] == 1:  # Draw crosses
                    pg.draw.lines(self.screen, color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                              height / BOARD_SIZE * (i + 0.5) - 10),
                                                             (width / BOARD_SIZE * (j + 0.5) + 10,
                                                              height / BOARD_SIZE * (i + 0.5) + 10)], 3)
                    pg.draw.lines(self.screen, color, True, [(width / BOARD_SIZE * (j + 0.5) - 10,
                                                              height / BOARD_SIZE * (i + 0.5) + 10),
                                                             (width / BOARD_SIZE * (j + 0.5) + 10,
                                                              height / BOARD_SIZE * (i + 0.5) - 10)], 3)
                elif self.board[i, j] == 2:  # Draw noughts
                    pg.draw.circle(self.screen, color,
                                   (width / BOARD_SIZE * (j + 0.5), height / BOARD_SIZE * (i + 0.5)), 12, 3)

        # drawing the next energy points of two players
        font1 = pg.font.Font(pg.font.get_default_font(), 16)
        font2 = pg.font.Font(pg.font.get_default_font(), 14)

        latest_player = self.current_player + 1 if self.current_player == 1 else 1
        # now print the text
        text_surface1 = font1.render('Energy Points:', True, pg.Color('black'))
        text_surface2 = font2.render(f'Player 1: {self.info["players"][1]["energy"]:.2f}', True, pg.Color('black'))
        text_surface3 = font2.render(f'Player 2: {self.info["players"][2]["energy"]:.2f}', True, pg.Color('black'))
        self.screen.blit(text_surface1, dest=(width + 15, 0))
        self.screen.blit(text_surface2, dest=(width + 15, 22))
        self.screen.blit(text_surface3, dest=(width + 15, 42))

        if latest_col >= 0 and latest_row >= 0:
            text_surface4 = font2.render(f'Player {latest_player} used {latest_enr * self.ener_bin_len:.2f} points', True,
                                         pg.Color('black'))
            self.screen.blit(text_surface4, dest=(width + 15, 72))

        board = np.transpose(
            np.array(pg.surfarray.pixels3d(self.screen)), axes=(1, 0, 2)
        )

        return board


In [6]:
def observation_and_action_constraint_splitter(obs):
    return obs['state'], obs['legal_moves']

## Random play test

In [7]:
def embed_mp4(filename):
    """Embeds an mp4 file in the notebook."""
    video = open(filename,'rb').read()
    b64 = base64.b64encode(video)
    tag = '''
    <video width="480" height="480" controls>
      <source src="data:video/mp4;base64,{0}" type="video/mp4">
    Your browser does not support the video tag.
    </video>'''.format(b64.decode())

    return IPython.display.HTML(tag)

def create_policy_eval_video(eval_env, policy, filename, fps=2):
    py_env = eval_env._envs[0]
    tf_env = eval_env
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
      time_step = tf_env.reset()
      video.append_data(py_env.render())
      while not time_step.is_last():
        action_step = policy.action(time_step, eval_env)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render())

    return embed_mp4(filename)


def create_policy_battle_video(eval_env, policy1, policy2, filename, fps=2):
    py_env = eval_env._envs[0]
    tf_env = eval_env
    filename = filename + ".mp4"
    with imageio.get_writer(filename, fps=fps) as video:
      time_step = tf_env.reset()
      video.append_data(py_env.render())
      while not time_step.is_last():
        action_step = policy1.action(time_step, eval_env)
        time_step = tf_env.step(action_step.action)
        video.append_data(py_env.render())
        if not time_step.is_last():
          action_step = policy2.action(time_step, eval_env)
          time_step = tf_env.step(action_step.action)
          video.append_data(py_env.render())

    return embed_mp4(filename)

def create_random_policy(train_env):
    return random_tf_policy.RandomTFPolicy(train_env.time_step_spec(), train_env.action_spec(),
                                           observation_and_action_constraint_splitter=observation_and_action_constraint_splitter)

In [8]:
py_env = TicTacToeEnv2()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
random_policy = create_random_policy(tf_env)
create_policy_eval_video(tf_env, random_policy, "random-agent-part2")

## Agent

In [9]:
def create_q_net(train_env):
  conv_layer_params = [32, 64, 128]
  action_tensor_spec = tensor_spec.from_spec(train_env.action_spec())
  num_actions = action_tensor_spec.maximum - action_tensor_spec.minimum + 1

  # Define a helper function to create Conv layers configured with the right
  # activation and kernel initializer.
  def conv_layer(num_units):
    return tf.keras.layers.Conv2D(
                        filters=num_units,
                        kernel_size=[3, 3],
                        padding="same",
                        data_format="channels_last",
                        activation=tf.nn.leaky_relu,
                        dtype=float)


  # QNetwork consists of a sequence of Conv layers followed by a dense layer
  # with `num_actions` units to generate one q_value per available action as
  # its output.
  normalization1 = tf.keras.layers.BatchNormalization()
  normalization2 = tf.keras.layers.BatchNormalization()
  conv_layers = [conv_layer(num_units) for num_units in conv_layer_params]
  action_conv = tf.keras.layers.Conv2D(filters=4,
                      kernel_size=[1, 1], padding="same",
                      data_format="channels_last")
  flatten = tf.keras.layers.Flatten()
  q_values_layer = tf.keras.layers.Dense(
      num_actions,
      activation=None,
      kernel_initializer=tf.keras.initializers.RandomUniform(
          minval=-0.03, maxval=0.03),
      bias_initializer=tf.keras.initializers.Constant(-0.2))
  q_net = sequential.Sequential([normalization1] + conv_layers + [action_conv, normalization2, flatten, q_values_layer])
  return q_net


In [10]:
def create_dqn_agent(train_env):
  optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
  q_net = create_q_net(train_env)
  agent = dqn_agent.DqnAgent(
      train_env.time_step_spec(),
      train_env.action_spec(),
      q_network=q_net,
      gamma=gamma,
      observation_and_action_constraint_splitter=observation_and_action_constraint_splitter,
      optimizer=optimizer,
      td_errors_loss_fn=common.element_wise_squared_loss,
      train_step_counter=tf.Variable(0))
  agent.initialize()
  return agent

In [11]:
import copy
import tensorflow as tf

from tf_agents.trajectories import PolicyStep
import numpy as np


class PlayPolicy():
    def __init__(self, policy):
        self.policy = policy

    def action(self, act: int, env=None):
        action_step = self.policy.action(act)
        if env:
            # when the board is empty, choose the center position.
            py_env = env._envs[0]
            if np.sum(py_env.board) == 0:
                return PolicyStep(action=tf.convert_to_tensor(np.array([40])))
            elif np.sum(py_env.board) > 6:
                current_player = py_env.current_player
                opponent = current_player + 1 if current_player == 1 else 1
                occupied_positions = py_env.info['Occupied']
                for act in range(81):
                    if act not in occupied_positions and self.drop_here_will_win(py_env, act, current_player):
                        return PolicyStep(action=tf.convert_to_tensor(np.array([act + (py_env.num_bin - 1) * 81])))
                for act in range(81):
                    if act not in occupied_positions and self.drop_here_will_win(py_env, act, opponent):
                        return PolicyStep(action=tf.convert_to_tensor(np.array([act + (py_env.num_bin - 1) * 81])))
        return action_step

    def drop_here_will_win(self, py_env, action, color):
        # check if four equal stones are aligned (horizontal, vertical or diagonal)
        directions = [[0, 1], [1, 0], [1, 1], [1, -1]]

        current_board = copy.deepcopy(py_env.board)
        r, c, _ = py_env.decode_action(action)
        current_board[r, c] = color

        for direct in directions:
            count = 0
            for offset in range(-3, 4):
                if 0 <= r + offset * direct[0] < 9 and 0 <= c + offset * direct[1] < 9:
                    if current_board[r + offset * direct[0], c + offset * direct[1]] == color:
                        count += 1
                        if count == 4:
                            return True
                    else:
                        count = 0

        return False


## Util functions

In [12]:
def compute_avg_return(environment, policy, num_episodes=10):

  total_return = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    episode_return = 0.0
    while not time_step.is_last():
      action_step = policy.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return += time_step.reward
    total_return += episode_return

  avg_return = total_return / num_episodes
  return avg_return.numpy()[0]

def compute_avg_return_battle(environment, policy1, policy2, num_episodes=10):

  total_return_1 = 0.0
  total_return_2 = 0.0
  for _ in range(num_episodes):
    episode_return_1 = 0.0
    episode_return_2 = 0.0
    time_step = environment.reset()
    while not time_step.is_last():
      action_step = policy1.action(time_step)
      time_step = environment.step(action_step.action)
      episode_return_1 += time_step.reward
      if not time_step.is_last():
        action_step = policy2.action(time_step)
        time_step = environment.step(action_step.action)
        episode_return_2 += time_step.reward
    total_return_1 += episode_return_1
    total_return_2 += episode_return_2

  avg_return_1 = total_return_1 / num_episodes
  avg_return_2 = total_return_2 / num_episodes
  return [avg_return_1.numpy()[0], avg_return_2.numpy()[0]]

def compute_avg_win_battle(environment, policy1, policy2, num_episodes=10):

  total_return_1 = 0.0
  total_return_2 = 0.0
  for _ in range(num_episodes):
    time_step = environment.reset()
    while not time_step.is_last():
      action_step = policy1.action(time_step)
      time_step = environment.step(action_step.action)
      if not time_step.is_last():
        action_step = policy2.action(time_step)
        time_step = environment.step(action_step.action)
    if environment._envs[0].get_result() == 1:
        total_return_1 += 1
    elif environment._envs[0].get_result() == 2:
        total_return_2 += 1

  avg_return_1 = total_return_1 / num_episodes
  avg_return_2 = total_return_2 / num_episodes
  return [avg_return_1, avg_return_2]

def create_zip_file(dirname, base_filename):
  return shutil.make_archive(base_filename, 'zip', dirname)

## Collect training data

In [13]:
def collect_episode(environment, agent1, agent2, replay_buffer):
    time_step = environment.reset()
    trajs_1, trajs_2 = [], []

    while not time_step.is_last():
        action_step = agent1.collect_policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        traj1 = trajectory.from_transition(time_step, action_step, next_time_step)
        trajs_1.append(traj1)
        time_step = next_time_step

        if not time_step.is_last():
            action_step = agent2.collect_policy.action(time_step)
            next_time_step = environment.step(action_step.action)
            traj2 = trajectory.from_transition(time_step, action_step, next_time_step)
            trajs_2.append(traj2)
            time_step = next_time_step

    change_flag = False
    # Modify the reward of each step according to the opponent's next step
    if len(trajs_1) == len(trajs_2):  # Player 2 won
        for i in range(len(trajs_1) - 1):
            trajs_1[i] = trajs_1[i].replace(reward=trajs_1[i].reward - trajs_2[i].reward)
            trajs_2[i] = trajs_2[i].replace(reward=trajs_2[i].reward - trajs_1[i + 1].reward)
        trajs_1[-1] = trajs_1[-1].replace(reward=trajs_1[-1].reward - trajs_2[-1].reward)
    else:  # Player 1 won
        change_flag = True
        for i in range(len(trajs_1) - 1):
            trajs_1[i] = trajs_1[i].replace(reward=trajs_1[i].reward - trajs_2[i].reward)
            trajs_2[i] = trajs_2[i].replace(reward=trajs_2[i].reward - trajs_1[i + 1].reward)

    for i in range(len(trajs_1)):
        replay_buffer.add_batch(trajs_1[i])
    for i in range(len(trajs_2)):
        replay_buffer.add_batch(trajs_2[i])

    # If the first player wins, then change the order in the next game.
    return change_flag

## Train the agent

In [14]:
def train_game_agent():
    train_py_env = TicTacToeEnv2(train=True)
    eval_py_env = TicTacToeEnv2()

    train_env = tf_py_environment.TFPyEnvironment(train_py_env)
    eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)

    agent1 = create_dqn_agent(train_env)
    agent2 = create_dqn_agent(train_env)

    play_policy = PlayPolicy(agent1.policy)
    random_policy = create_random_policy(train_env)

    replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
        data_spec=agent1.collect_data_spec,
        batch_size=train_env.batch_size,
        max_length=replay_buffer_max_length)

    for _ in range(initial_collect_episodes):
        collect_episode(train_env, agent1, agent2, replay_buffer)

    dataset = replay_buffer.as_dataset(
        num_parallel_calls=3, sample_batch_size=batch_size,
        num_steps=n_step_update + 1).prefetch(3)
    iterator = iter(dataset)

    policy_dir = os.path.join(tempdir, 'play_policy_part2')
    tf_policy_saver = policy_saver.PolicySaver(play_policy.policy)

    # (Optional) Optimize by wrapping some of the code in a graph using TF function.
    agent1.train = common.function(agent1.train)
    agent2.train = common.function(agent2.train)

    # Reset the train step.
    agent1.train_step_counter.assign(0)
    agent2.train_step_counter.assign(0)
    policy_win_rate = compute_avg_win_battle(eval_env, play_policy, random_policy, num_eval_episodes)[0]
    print('Before training: 1_win = {0}'.format(policy_win_rate))

    bst = 0

    # Reset the environment.
    train_env.reset()

    x, y, change_flag = agent1, agent2, False
    for idx in range(num_iterations):
        # Collect a few episodes using collect_policy and store the transitions to the replay buffer.
        for _ in range(collect_episodes_per_iteration):
            if change_flag:
                x, y = y, x
            change_flag = collect_episode(train_env, x, y, replay_buffer)

        # Sample a batch of data from the buffer and update the agent's network.
        experience, unused_info = next(iterator)
        train_loss1 = agent1.train(experience).loss
        experience, unused_info = next(iterator)
        agent2.train(experience)

        step = agent1.train_step_counter.numpy()

        if step % log_interval == 0:
            print('step = {0}: loss1 = {1}'.format(step, train_loss1))

        if step % eval_interval == 0:
            policy_win_rate1 = compute_avg_win_battle(eval_env, play_policy, random_policy, num_eval_episodes)[0]
            policy_win_rate2 = compute_avg_win_battle(eval_env, random_policy, play_policy, num_eval_episodes)[1]
            print('Evaluation (step = {0}): offense_win = {1}, defense_win = {2}'.format(step,
                                                                                         policy_win_rate1,
                                                                                         policy_win_rate2,
                                                                                         ))
            policy_win_rate = (policy_win_rate1 + policy_win_rate2) / 2
            if policy_win_rate >= bst:
                bst = policy_win_rate
                tf_policy_saver.save(policy_dir)
                # create_zip_file(policy_dir, os.path.join(tempdir, 'exported_policy_part2'))

In [ ]:
train_game_agent()

Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Before training: 1_win = 0.58


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


step = 5: loss1 = 0.5202822685241699
step = 10: loss1 = 0.3848797082901001
step = 15: loss1 = 0.27704811096191406
step = 20: loss1 = 0.30223172903060913


Evaluation (step = 20): offense_win = 0.46, defense_win = 0.62


...

step = 285: loss1 = 0.3077732026576996
step = 290: loss1 = 0.48336368799209595
step = 295: loss1 = 0.36547625064849854
step = 300: loss1 = 0.35242414474487305
Evaluation (step = 300): offense_win = 0.68, defense_win = 0.58
step = 305: loss1 = 0.3302222490310669
step = 310: loss1 = 0.34526312351226807
step = 315: loss1 = 0.4362480342388153
step = 320: loss1 = 0.2864343523979187
Evaluation (step = 320): offense_win = 0.64, defense_win = 0.68
step = 325: loss1 = 0.39909613132476807
step = 330: loss1 = 0.5082756280899048
step = 335: loss1 = 0.5163301825523376
step = 340: loss1 = 0.23449759185314178
Evaluation (step = 340): offense_win = 0.76, defense_win = 0.68
step = 345: loss1 = 0.7989844083786011
step = 350: loss1 = 0.2730916738510132
step = 355: loss1 = 0.3853558897972107
step = 360: loss1 = 0.5267512202262878
Evaluation (step = 360): offense_win = 0.62, defense_win = 0.66
step = 365: loss1 = 0.36465027928352356
step = 370: loss1 = 0.5887670516967773
step = 375: loss1 = 0.5837392807006

/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 385: loss1 = 0.6099482774734497
step = 390: loss1 = 0.6423933506011963
step = 395: loss1 = 0.6420032978057861
step = 400: loss1 = 0.1340738832950592
Evaluation (step = 400): offense_win = 0.74, defense_win = 0.66
step = 405: loss1 = 0.589032769203186
step = 410: loss1 = 0.36881357431411743
step = 415: loss1 = 0.28935325145721436
step = 420: loss1 = 0.28610315918922424
Evaluation (step = 420): offense_win = 0.86, defense_win = 0.76


/usr/local/lib/python3.10/dist-packages/tensorflow/python/saved_model/nested_structure_coder.py:458: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


step = 425: loss1 = 0.39218997955322266
step = 430: loss1 = 0.41472870111465454
step = 435: loss1 = 0.30489248037338257
step = 440: loss1 = 0.23932865262031555


## Evaluate the trained model

In [ ]:
eval_py_env = TicTacToeEnv2()
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)
random_policy = random_policy = create_random_policy(eval_env)
policy_dir = os.path.join(tempdir, 'play_policy_part2')

play_policy = PlayPolicy(tf.saved_model.load(policy_dir))

#### Trained agent (X) v.s. Random policy (O)

In [ ]:
create_policy_battle_video(eval_env, play_policy, random_policy, 'trained-agent-2-with-random')

#### Random policy (O) v.s. Trained agent (X)

In [ ]:
create_policy_battle_video(eval_env, random_policy, play_policy, 'random-with-trained-agent-2')

#### Trained agent (X) v.s. Trained agent (O)

In [ ]:
create_policy_battle_video(eval_env, play_policy, play_policy, 'trained-agent-2-self-battle')